# 이웃 기반 협업 필터링 구현


*   이웃 기반 협업 필터링에서 유사도 계산 -> 자카드, 피어슨, 코사인


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import pandas as pd
import numpy as np
from math import sqrt
from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [7]:
path = '/content/drive/MyDrive/Colab Notebooks/추천 시스템/data/movielens'

ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'))

In [8]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=1234)

print(train_df.shape, test_df.shape)

(80668, 4) (20168, 4)


In [9]:
train_df

,userId,movieId,rating,timestamp
95713,600,5943,3.0,1237714356
61560,407,2571,5.0,1424349171
77204,482,8958,4.0,1105397126
93367,599,2322,2.5,1498515283
90892,590,2959,3.5,1258416553
...,...,...,...,...
89460,580,1923,4.0,1167790046
60620,391,2232,4.0,1030826940
34086,232,4344,4.0,1206995838
58067,380,166528,5.0,1493419871


In [24]:
sparse_matrix = train_df.groupby('movieId').apply(lambda x: pd.Series(x['rating'].values, index=x['userId'])).unstack()
print(sparse_matrix) # !!이런 형태의 매트리스를 스파스 매트리스라고 한다!!

userId   1    2    3    4    5    6    7    ...  604  605  606  607  608  609  610
movieId                                     ...                                   
1        NaN  NaN  NaN  NaN  4.0  NaN  4.5  ...  3.0  4.0  2.5  NaN  2.5  NaN  5.0
2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  3.5  NaN  NaN  2.0  NaN  NaN
3        4.0  NaN  NaN  NaN  NaN  5.0  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN
4        NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN
5        NaN  NaN  NaN  NaN  NaN  5.0  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
193573   NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN
193579   NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN
193581   NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN
193587   NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN
1936

In [25]:
sparse_matrix = sparse_matrix.fillna(0)

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

def cossim_matrix(a, b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns=a.index.values, index=a.index)

    return cossim_df

### 이웃 기반 협업 필터링 추천 점수 계산

In [30]:
item_cossim_df =  cossim_matrix(sparse_matrix, sparse_matrix)
item_cossim_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184257,184349,184471,184721,184791,184931,184987,184997,185033,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188675,188797,189043,189111,189333,189547,190183,190207,190209,190213,190215,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.304336,0.267816,0.040259,0.221228,0.266544,0.149392,0.132943,0.182044,0.296838,0.257709,0.139632,0.113701,0.164856,0.103301,0.242000,0.246986,0.101015,0.251669,0.131482,0.261433,0.182307,0.164879,0.222668,0.266991,0.140973,0.108473,0.109745,0.181797,0.053777,0.162647,0.422169,0.402208,0.266803,0.116007,0.297235,0.0,0.085053,0.021893,0.114641,...,0.000000,0.057923,0.045449,0.028962,0.090051,0.077231,0.077231,0.077231,0.077231,0.045207,0.0,0.05933,0.040932,0.0,0.060720,0.082251,0.028962,0.028962,0.061785,0.000000,0.077231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.304336,1.000000,0.226138,0.052482,0.154783,0.209716,0.189420,0.068012,0.027945,0.303157,0.244600,0.156918,0.073411,0.083439,0.158038,0.205833,0.146949,0.193136,0.458342,0.092049,0.235169,0.142401,0.052102,0.182855,0.057490,0.139123,0.037172,0.115826,0.189215,0.000000,0.152457,0.258012,0.324845,0.147365,0.119390,0.336617,0.0,0.078364,0.029729,0.092547,...,0.000000,0.000000,0.165209,0.000000,0.050024,0.000000,0.000000,0.000000,0.000000,0.156683,0.0,0.00000,0.000000,0.0,0.069710,0.136606,0.000000,0.000000,0.000000,0.110117,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.267816,0.226138,1.000000,0.000000,0.306435,0.245555,0.368724,0.168267,0.253679,0.178219,0.235708,0.122641,0.149451,0.138664,0.052048,0.236722,0.188948,0.203784,0.244938,0.073150,0.229087,0.223028,0.115734,0.247492,0.198245,0.127082,0.238175,0.114782,0.110742,0.000000,0.198132,0.233039,0.151203,0.239877,0.119628,0.191147,0.0,0.123729,0.103266,0.208362,...,0.000000,0.000000,0.000000,0.000000,0.089110,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.096418,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.040259,0.052482,0.000000,1.000000,0.095673,0.068708,0.205962,0.000000,0.000000,0.044835,0.144579,0.000000,0.000000,0.161852,0.000000,0.000000,0.125440,0.000000,0.025580,0.000000,0.155352,0.069693,0.000000,0.000000,0.185360,0.051463,0.000000,0.000000,0.000000,0.000000,0.187215,0.050263,0.064871,0.112260,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.221228,0.154783,0.306435,0.095673,1.000000,0.238683,0.343507,0.204088,0.222925,0.150729,0.215188,0.198332,0.110459,0.153343,0.000000,0.156847,0.226190,0.128029,0.194406,0.046290,0.125510,0.176078,0.117994,0.246658,0.267305,0.089126,0.260605,0.049500,0.163221,0.000000,0.275988,0.222996,0.181409,0.198536,0.169278,0.182216,0.0,0.083955,0.000000,0.240221,...,0.178434,0.000000,0.000000,0.000000,0.054040,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.058472,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [43]:
userId_grouped = train_df.groupby('userId')
item_prediction_result_df = pd.DataFrame(index=list(userId_grouped.indices.keys()), columns=sparse_matrix.index)

In [44]:
for userId, group in tqdm(userId_grouped):
    # print(userId)
    user_sim = item_cossim_df.loc[group['movieId']]
    # print(user_sim)

    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum +1)
    item_prediction_result_df.loc[userId] = pred_ratings

print(item_prediction_result_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.



movieId   1        2        3       ...    193581    193587    193609
1        4.20279  4.18783  4.17609  ...         0         0   1.38273
2        3.19261  3.15207  2.46138  ...   1.70885   1.70885   2.32666
3        1.32315  1.20446  1.39635  ...         0         0         0
4        3.48994  3.48805  3.38864  ...  0.505535  0.505535     1.963
5        3.27916   3.1278  2.99947  ...         0         0  0.562705
..           ...      ...      ...  ...       ...       ...       ...
606       3.6229  3.54834   3.4441  ...   1.39051   1.39051   2.99518
607      3.66544  3.65456  3.65215  ...         0         0   0.68735
608      3.19053  3.14796  2.95741  ...   0.44408   0.44408   3.46138
609      2.90968  2.85971   2.7548  ...         0         0  0.272122
610      3.77235  3.75405  3.77113  ...   3.46181   3.46181   3.80464

[610 rows x 8938 columns]


In [47]:
user_sparse_matrix = sparse_matrix.fillna(0).T

In [48]:
user_cossim_df = cossim_matrix(user_sparse_matrix, user_sparse_matrix)
user_cossim_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.016665,0.070570,0.160438,0.075410,0.087404,0.123664,0.069225,0.037416,0.009900,0.116869,0.010900,0.097073,0.097351,0.160905,0.130733,0.233488,0.159436,0.229175,0.106378,0.134026,0.042174,0.080940,0.118462,0.057098,0.074305,0.176682,0.154508,0.115075,0.095219,0.116415,0.096891,0.116097,0.060039,0.018472,0.033715,0.080014,0.103986,0.265307,0.062876,...,0.068639,0.092376,0.193911,0.059231,0.038337,0.027807,0.259536,0.000000,0.127610,0.196391,0.032494,0.060829,0.041263,0.061662,0.118403,0.065398,0.079916,0.152958,0.049672,0.254597,0.084700,0.103389,0.142405,0.099736,0.068952,0.104889,0.232882,0.014377,0.226264,0.208385,0.056163,0.124896,0.162324,0.050852,0.101583,0.128198,0.240652,0.225897,0.063984,0.095228
2,0.016665,1.000000,0.000000,0.004295,0.020560,0.030996,0.027726,0.000000,0.000000,0.057112,0.050529,0.000000,0.000000,0.019706,0.094221,0.042154,0.080419,0.157239,0.014791,0.016695,0.076210,0.079124,0.000000,0.111357,0.181943,0.000000,0.000000,0.033860,0.096737,0.066817,0.000000,0.021255,0.038112,0.021178,0.000000,0.070899,0.034344,0.030872,0.000000,0.020290,...,0.000000,0.039103,0.124467,0.040580,0.000000,0.000000,0.000000,0.000000,0.000000,0.050032,0.130993,0.149300,0.000000,0.000000,0.094519,0.080066,0.000000,0.000000,0.032529,0.050973,0.042262,0.036289,0.053546,0.015933,0.000000,0.095991,0.014417,0.000000,0.079978,0.028680,0.151666,0.019379,0.006645,0.000000,0.000000,0.028180,0.000000,0.046286,0.033522,0.090288
3,0.070570,0.000000,1.000000,0.002677,0.000000,0.003477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017739,0.037489,0.003184,0.013792,0.015762,0.007023,0.003222,0.000488,0.003862,0.003162,0.000000,0.000000,0.002843,0.012381,0.007261,0.000000,0.004728,0.015895,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002246,0.003035,...,0.009987,0.007018,0.000000,0.000000,0.000000,0.037043,0.003330,0.000000,0.000000,0.022490,0.005126,0.000000,0.000000,0.000000,0.000000,0.000000,0.005185,0.000000,0.000000,0.025883,0.000000,0.000000,0.003586,0.002482,0.000000,0.011629,0.019767,0.000000,0.030780,0.003817,0.002878,0.002174,0.027609,0.000000,0.000000,0.012090,0.000000,0.023927,0.000000,0.018332
4,0.160438,0.004295,0.002677,1.000000,0.121648,0.093634,0.106495,0.047930,0.000000,0.034281,0.051343,0.062434,0.068593,0.047313,0.056951,0.153868,0.115363,0.108863,0.173788,0.092159,0.035909,0.053011,0.069875,0.055751,0.021803,0.073772,0.100712,0.125315,0.062167,0.059498,0.075626,0.177156,0.134276,0.036773,0.050788,0.091729,0.072888,0.054797,0.169479,0.043322,...,0.011451,0.144048,0.102033,0.069941,0.009641,0.007079,0.129647,0.000000,0.063585,0.122387,0.047020,0.055886,0.058342,0.033610,0.095249,0.047437,0.105421,0.037728,0.051045,0.184120,0.105999,0.049010,0.209392,0.062336,0.082067,0.093637,0.210657,0.019309,0.160776,0.186949,0.065328,0.116656,0.251105,0.056396,0.075940,0.187236,0.102819,0.109515,0.038805,0.086941
5,0.075410,0.020560,0.000000,0.121648,1.000000,0.164390,0.075932,0.302418,0.000000,0.000000,0.118213,0.053365,0.000000,0.135476,0.083971,0.063830,0.096110,0.095612,0.079224,0.032121,0.041732,0.019271,0.052776,0.076803,0.052181,0.091675,0.102966,0.090528,0.028561,0.000000,0.017516,0.232930,0.204039,0.038300,0.296278,0.030812,0.214088,0.195768,0.106027,0.173635,...,0.000000,0.057779,0.068374,0.189874,0.000000,0.000000,0.088809,0.000000,0.159289,0.072059,0.075021,0.000000,0.061152,0.223592,0.000000,0.029744,0.127100,0.256836,0.096130,0.085791,0.000000,0.209454,0.146134,0.093192,0.085930,0.085811,0.068642,0.000000,0.054620,0.101157,0.090020,0.306634,0.103646,0.145354,0.119800,0.071333,0.064705,0.111755,0.159946,0.049236
...,..

In [49]:
movieId_grouped = train_df.groupby('movieId')
movieId_prediction_result_df = pd.DataFrame(index=list(movieId_grouped.indices.keys()), columns=sparse_matrix.index)

In [53]:
for movieId, group in tqdm(movieId_grouped):
    user_sim = user_cossim_df.loc[group['userId']]
    user_ranking = group['rating']
    sim_sum = user_sim.sum(axis=0)
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_ranking) / (sim_sum +1)
    movieId_prediction_result_df.loc[movieId] = pred_ratings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [54]:
movieId_prediction_result_df

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184257,184349,184471,184721,184791,184931,184987,184997,185033,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188675,188797,189043,189111,189333,189547,190183,190207,190209,190213,190215,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
1,3.67043,3.34895,2.121,3.61963,3.76122,3.83087,3.77535,3.79756,3.17211,3.53956,3.77271,3.25295,3.38608,3.71767,3.67586,3.62024,3.68222,3.71063,3.7608,3.65439,3.7096,3.54082,3.46236,3.65635,3.54933,3.69778,3.70017,3.63817,3.57472,3.60583,3.69811,3.77944,3.60127,3.32949,3.73684,3.66265,3.70991,3.59034,3.71246,3.86354,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.01898,2.57281,1.27956,2.93753,3.21044,3.32198,3.10961,3.3381,2.35781,2.8281,3.19533,2.41644,2.58414,3.20225,3.05923,2.95389,3.08068,3.12762,3.13179,3.00293,3.13079,2.90958,2.70137,2.99499,2.76909,3.27871,3.13508,3.05045,2.94201,2.87687,3.01183,3.1045,2.98701,2.49345,3.30884,2.98861,3.26457,2.95002,3.13921,3.37255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.02703,1.7197,1.26626,2.73966,2.92431,3.32667,2.83467,3.12298,1.68523,2.16925,3.03148,2.02516,2.12632,3.09258,2.77223,2.6048,2.84658,2.88318,3.04508,2.66741,2.69214,2.44399,2.4074,2.72977,1.9996,3.0305,2.85355,2.86627,2.57443,2.13558,2.86902,3.03456,2.62894,2.12788,3.08393,2.79625,2.99991,2.66993,3.14766,3.30759,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.636151,0.146062,0.0367179,0.72914,0.957915,1.21591,0.572114,1.16575,0.0990461,0.172367,1.01382,0.208322,0.191233,1.64497,0.439262,0.673211,0.758475,0.735548,0.754045,0.258811,0.344197,0.261046,0.379184,0.460082,0.161154,1.12963,0.574072,0.610436,0.491937,0.203056,0.686023,1.09784,0.410265,0.377929,1.20179,0.616147,1.1298,0.41386,0.935171,1.08397,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.33427,1.55944,0.466202,2.1024,2.54943,3.03136,2.35059,2.71027,1.19739,1.72931,2.67806,1.62483,1.57144,2.69429,2.21885,2.06364,2.31724,2.364,2.51215,1.95871,2.198,1.87701,1.82359,2.23057,1.66264,2.61757,2.29546,2.26701,2.00612,1.92448,2.61819,2.70976,2.07908,1.75008,2.76508,2.21247,2.66091,1.97148,2.54273,3.02431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,0,0.379994,0,0.046584,0,0,0.0363844,0,0,0.189819,0,0,0,0,0.210615,0,0.189158,0.393961,0,0.0508683,0.44719,0.144829,0.0448231,0.057173,0.285037,0,0,0.136796,0.116374,0.408297,0,0,0.0800611,0,0,0,0,0.145893,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193579,0,0.332495,0,0.040761,0,0,0.0318363,0,0,0.166092,0,0,0,0,0.184288,0,0.165513,0.344716,0,0.0445097,0.391291,0.126725,0.0392202,0.0500264,0.249408,0,0,0.119696,0.101827,0.35726,0,0,0.0700535,0,0,0,0,0.127656,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193581,0,0.379994,0,0.046584,0,0,0.0363844,0,0,0.189819,0,0,0,0,

In [55]:
movieId_prediction_result_df[6].sort()

1           3.83087
2           3.32198
3           3.32667
4           1.21591
5           3.03136
            ...    
193573            0
193579            0
193581            0
193587            0
193609    0.0660016
Name: 6, Length: 8938, dtype: object

### RMSE로 추천시스템 성능 평가하기

In [56]:
test_df

,userId,movieId,rating,timestamp
99731,610,3527,5.0,1479545223
97583,606,1250,3.5,1171376891
38197,262,213,5.0,840310907
11474,68,69406,3.0,1261622505
34105,232,4728,3.0,1218166950
...,...,...,...,...
41080,279,593,4.0,1506394242
4897,31,780,4.0,850466616
8023,56,410,3.0,835799188
77467,483,2291,4.0,1415579167


In [63]:
groups_with_movie_ids = test_df.groupby(by='movieId')
intersection_movie_df = sorted(list(set(movieId_prediction_result_df.columns) - set(groups_with_movie_ids.indices.keys())))
intersection_movie_df = sorted(list(set(movieId_prediction_result_df.columns) - set(groups_with_movie_ids.indices.keys())))

In [81]:
def evalute(test_df, prediction_result_df):
    groups_with_movie_ids = test_df.groupby(by='movieId')
    groups_with_user_ids = test_df.groupby(by='userId')
    intersection_movie_ids = sorted(list(set(prediction_result_df.columns) - set(groups_with_movie_ids.indices.keys())))
    intersection_user_ids = sorted(list(set(prediction_result_df.columns) - set(groups_with_user_ids.indices.keys())))

    compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_movie_ids]
    
    # test_df에 대해서 RMSE 계산
    grouped = test_df.groupby(by='userId')
    result_df = pd.DataFrame(columns=['rmse'])

    for userId, group in tqdm(grouped):
        if userId in intersection_user_ids:
            pred_ratings = compressed_prediction_df.loc[userId][compressed_prediction_df.loc[userId].index.intersection(list(group['movieId'].values))]
            pred_ratings = pred_ratings.to_frame(name='rating').reset_index().rename(columns={'index':'movieId','rating':'pred_rating'})
            actual_ratings = group[['rating', 'movieId']].rename(columns={'rating':'actual_rating'})
            final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['movieId'])
            final_df = final_df.round(4) # 반올림
            print(final_df)
          # if not final_df.empty:
          #     rmse = sqrt(mean_squared_error(final_df['rating_actual'], final_df['rating_pred']))
          #     result_df.loc[userId] = rmse
          #     # print(userId, rmse)
    return final_df


In [82]:
evalute(test_df, movieId_prediction_result_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


UnboundLocalError: ignored